In [42]:
import cv2
import numpy as np

In [54]:
def ball_tracker(frame):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    template = cv2.imread("ball.png", 0)
    h,w = template.shape

    methods = [cv2.TM_CCOEFF, cv2.TM_CCOEFF_NORMED, cv2.TM_CCORR, cv2.TM_CCORR_NORMED, cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]
    method = methods[3]
    img2 = image.copy()
    result = cv2.matchTemplate(img2, template, cv2.TM_CCOEFF_NORMED)
    minv, maxv, minl, maxl = cv2.minMaxLoc(result)
    if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
        location = minl
    else:
        location = maxl

    bottom_right = (location[0] + w , location[1] + h)
    cv2.rectangle(frame, location, bottom_right, (162,200,146), -1)
    return frame

        
    

In [ ]:
# Load video
video_path = "volleyball_match.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create VideoWriter object
output_path = "volleyball_output.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4 format
out = cv2.VideoWriter(output_path, fourcc, frame_rate, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert frame to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Define HSV range for team jerseys (adjust as needed)
    lower_team1 = np.array([5, 0, 100])  # Range for yellow jerseys
    upper_team1 = np.array([67, 255, 255])
    
    lower_team2 = np.array([176, 170, 100])  # Range for red jerseys
    upper_team2 = np.array([183, 240, 240])
    
    # Create masks
    mask_team1 = cv2.inRange(hsv, lower_team1, upper_team1)
    mask_team2 = cv2.inRange(hsv, lower_team2, upper_team2)
    
    # Find contours for both teams
    contours_team1, _ = cv2.findContours(mask_team1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_team2, _ = cv2.findContours(mask_team2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    team1_count = 0
    team2_count = 0
    
    for contour in contours_team1:
        if cv2.contourArea(contour) > 1000:  # Adjust threshold to filter noise
            x, y, w, h = cv2.boundingRect(contour)
            team1_count += 1
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)  # Blue for Team 1
    
    for contour in contours_team2:
        if cv2.contourArea(contour) > 1000:  # Adjust threshold to filter noise
            x, y, w, h = cv2.boundingRect(contour)
            team2_count += 1
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)  # Red for Team 2
    
    # Apply ball tracking
    frame = ball_tracker(frame)
    
    # Display player count
    cv2.putText(frame, f"Team 1: {team1_count}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(frame, f"Team 2: {team2_count}", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    # Write frame to output video
    out.write(frame)
    
    # Show frame
    cv2.imshow("Volleyball Player Counting", frame)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Output video saved as {output_path}")